# 한-영 번역기
한-영 번역기에 오신 것을 환영합니다!

In [1]:
from transformers import (
    EncoderDecoderModel,
    GPT2Tokenizer,
)

from lib.tokenization_kobert import KoBertTokenizer

In [3]:
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
trg_tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')

model = EncoderDecoderModel.from_pretrained('dump/best_model')
model.eval()
# model.cuda()
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


EncoderDecoderModel(
  (encoder): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(

In [6]:
from IPython.display import display
import ipywidgets as widgets

kor = widgets.Textarea(
    placeholder='번역할 한국어',
    description="입력",
    disabled=False
)

button = widgets.Button(
    description='번역!',
    disabled=False,
    tooltip='해당 한국어 입력을 번역합니다.'
)

eng = widgets.Textarea(
    description="출력",
    disabled=True
)

def translate(_):
    eng.value = ""
    text = kor.get_interact_value()
    embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
    # embeddings = {k: v.cuda() for k, v in embeddings.items()}
    output = model.generate(**embeddings)[0, 1:-1]
    eng.value = trg_tokenizer.decode(output.cpu())

button.on_click(translate)
display(kor, button, eng)

Textarea(value='', description='입력', placeholder='번역할 한국어')

Button(description='번역!', style=ButtonStyle(), tooltip='해당 한국어 입력을 번역합니다.')

Textarea(value='', description='출력', disabled=True)